# Debug Bert notebook
- [x] trivial case, with predicting shifted tokens, instead of mlm.
- [x] trivial case, of classifying presence of marker token, at a random position.
- [ ] Normalize frequency of labels across training and test samples

In [1]:
%load_ext autoreload
%autoreload 2

import tokentango

TokenTango


In [2]:
import torch
from torch.optim import Adam, AdamW
from torch import nn, functional as F
from plotly import express as px
import plotly.graph_objects as go
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from matplotlib import pyplot as plt
import itertools
import math
import datetime as dt

In [3]:
print(f"CUDA available: {torch.cuda.is_available()}")
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs: {num_gpus}")
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
    print(f"  CUDA Capability: {torch.cuda.get_device_properties(i).major}.{torch.cuda.get_device_properties(i).minor}")

CUDA available: True
Number of GPUs: 2
GPU 0: NVIDIA GeForce GTX 1080
  Memory: 8.51 GB
  CUDA Capability: 6.1
GPU 1: NVIDIA GeForce GTX 1050 Ti
  Memory: 4.23 GB
  CUDA Capability: 6.1


In [4]:
device = torch.device("cuda:0")

```py
# 1. Embeddings padding_idx = 0, might conflict with cls, or other tokens? (model.py)
# 2. xs, masks and cls classes are all made with seperate calls to train_test_split (in bert_from_scratch.py)
#    so maybe the cls' dont match the text
num_samples = 2000

xs = torch.randint(low=1, high=10, size=(num_samples,5), dtype=torch.int32)
xs = torch.cat([torch.zeros(num_samples, 1, dtype=torch.int), xs], dim=1).to(device)

ys = xs.clone()
cls_label = [float(any(n == 1 for n in xs[i,:])) for i in range(num_samples)]

split_at = int(0.2 * num_samples)

train_x = xs[:split_at, :]
train_y = ys[:split_at, :]
train_cls = cls_label[:split_at]

test_x = xs[split_at:, :]
test_y = ys[split_at:, :]
test_cls = cls_label[split_at:]
```

In [ ]:
#train_x = tokentango.fake_news.train_mlm
#train_y = tokentango.fake_news.train_x
#train_cls = tokentango.fake_news.train_y
train_x, train_y, train_cls, test_x, test_y, test_cls = tokentango.fake_news.load_data(0.8)

/home/bigbox/Projects/tokentango/src/tokentango/fake_news.py:15: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  large_set = pd.read_csv("995,000_rows.csv").sample(frac=frac).reset_index(drop=True)





new_labels
reliable    330584
fake        166071
Name: count, dtype: int64


/home/bigbox/Projects/tokentango/src/tokentango/fake_news.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  large_set = large_set.groupby('new_labels').apply(lambda x: x.sample(min_count, random_state=42)).sample(frac=1).reset_index(drop=True)


0.5


In [ ]:
train_x = train_x.to(device)
train_y = train_y.to(device)
train_cls = train_cls.to(device)

test_x = torch.tensor(test_x).to(device)
test_y = torch.tensor(test_y).to(device)
test_cls = torch.tensor(test_cls).to(device)

In [ ]:
size_in_bytes = train_x.element_size() * train_x.numel() + train_y.element_size() * train_y.numel() + train_cls.element_size() * train_cls.numel()
size_in_bytes2 = test_x.element_size() * test_x.numel() + test_y.element_size() * test_y.numel() + test_cls.element_size() * test_cls.numel()
size_in_gb = (size_in_bytes + size_in_bytes2) / (1024 ** 3)

print(f"Tensor size: {size_in_gb:.4f} GB")

In [ ]:
# Current memory usage
print(torch.cuda.memory_allocated() / 1024**3, "GB allocated")
print(torch.cuda.memory_reserved() / 1024**3, "GB reserved by caching allocator")

# Optional: summary of tensors by type
from torchsummary import summary

# Or more advanced: use torch.cuda.memory_summary
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
model = tokentango.BertClassifier(300, 40000, device).to(device)

In [ ]:
%%time

# 1. embedding padding_idx (from model.py) should probably be set to pad_token_id (from fake_news.py)
# 3. Maybe change named_parameters to parameters? (also in bert_from_scratch.py)
#optimizer = AdamW(model.parameters(), lr = 1e-4, eps = 1e-8)

result = tokentango.train.train(model, train_x, train_y, train_cls, test_x, test_y, test_cls, device)

In [15]:
np.mean([tokentango.train.test_accuracy(model, test_x, test_y, test_cls, device) for _ in range(5)])

np.float64(84.9171936163806)

In [ ]:
px.line(mlm_losses)

In [ ]:
px.line(cls_losses)

In [ ]:
px.line(test_acc)

In [ ]:
correct = 0
for idx in range(len(train_cls)):
    x = train_y[idx:idx+1,:]
    hidden = model.hidden(x)
    output = model.classify(hidden)
    #print(output.cpu().detach().item(), cls_label[idx:idx+1], x.cpu().detach().tolist())
    #correct += int((output.cpu().detach().item()<0.5) == bool(np.signbit(cls_label[idx:idx+1][0])))
    correct += int(np.sign(output.cpu().detach().item()) == np.sign(train_cls[idx].cpu()))

accuracy = correct / len(test_cls)*100
print(f'{accuracy}%')

In [ ]:
outputs = []
num_samples = len(test_cls)

model.eval()
with torch.no_grad():
    for idx in range(0, num_samples):
        x = train_x[idx:idx+1,:]
        hidden = model.hidden(x)
        output = model.classify(hidden)
        outputs.append(output)

In [ ]:
outputs = torch.sign(torch.cat(outputs))

predicted_values = torch.round(outputs)
predicted_values = predicted_values.cpu().view(-1).numpy()
true_values = test_cls.cpu().numpy()
label_values = ["reliabel", "fake"]

test_accuracy = np.sum(predicted_values == true_values) / len(true_values)
print ("Test Accuracy:", test_accuracy)

print(classification_report(true_values, predicted_values, target_names=[str(l) for l in label_values]))

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# In[32]:


cm_test = confusion_matrix(true_values, predicted_values)

np.set_printoptions(precision=2)

#plt.figure(figsize=(6,6))
#plot_confusion_matrix(cm_test, classes=label_values, title='Confusion Matrix - Test Dataset')
plt.figure(figsize=(6,6))
plot_confusion_matrix(cm_test, classes=label_values, title='Confusion Matrix - Test Dataset', normalize=True)

In [ ]:
print(sum(n == 1 for n in predicted_values))
print(sum(n == -1 for n in predicted_values))

In [ ]:
print(sum(n == 1 for n in train_cls))
print(sum(n == -1 for n in train_cls))
print(sum(n == 1 for n in test_cls))
print(sum(n == -1 for n in test_cls))